In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as geopd

## Data Informations
All data it's from 
[CapitalBikeSahre](https://www.capitalbikeshare.com/system-data)


In [8]:
dataframe_stations = pd.read_json('https://gbfs.capitalbikeshare.com/gbfs/en/station_information.json', orient='values')
dataframe_stations = pd.DataFrame(dataframe_stations.data.stations).set_index('legacy_id')
print('Dataframe Length: ',len(dataframe_stations))
dataframe_stations.head()

Dataframe Length:  644


,station_id,has_kiosk,rental_uris,name,lon,capacity,short_name,lat,station_type,electric_bike_surcharge_waiver,rental_methods,external_id,region_id,eightd_has_key_dispenser,eightd_station_services
legacy_id,,,,,,,,,,,,,,,
1,1,True,{'android': 'https://dc.lft.to/lastmile_qr_sca...,Eads St & 15th St S,-77.053230,15,31000,38.858971,classic,False,"[CREDITCARD, KEY]",082469cc-1f3f-11e7-bf6b-3863bb334450,41,False,[]
2,2,True,{'android': 'https://dc.lft.to/lastmile_qr_sca...,18th St & S Eads St,-77.053320,11,31001,38.857250,classic,False,"[CREDITCARD, KEY]",08246b69-1f3f-11e7-bf6b-3863bb334450,41,False,[]
3,3,True,{'android': 'https://dc.lft.to/lastmile_qr_sca...,Crystal Dr & 20th St S,-77.049232,17,31002,38.856425,classic,False,"[CREDITCARD, KEY]",08246c35-1f3f-11e7-bf6b-3863bb334450,41,False,[]
4,4,True,{'android': 'https://dc.lft.to/lastmile_qr_sca...,Crystal Dr & 15th St S,-77.049417,16,31003,38.861056,classic,False,"[CREDITCARD, KEY]",08246cd5-1f3f-11e7-bf6b-3863bb334450,41,False,[]
5,5,True,{'android': 'https://dc.lft.to/lastmile_qr_sca...,Aurora Hills Cmty Ctr / 18th St & S Hayes St,-77.059490,12,31004,38.857866,classic,False,"[CREDITCARD, KEY]",08246d68-1f3f-11e7-bf6b-3863bb334450,41,False,[]


In [3]:
dataframe_tripdataq1 = pd.read_csv('2016Q1-capitalbikeshare-tripdata.csv')
print('Dataframe Length: ',len(dataframe_tripdataq1))
dataframe_tripdataq1.head()

Dataframe Length:  552399


,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
0,166,2016-01-01 00:06:58,2016-01-01 00:09:44,31102,11th & Kenyon St NW,31105,14th & Harvard St NW,W01346,Member
1,448,2016-01-01 00:10:20,2016-01-01 00:17:48,32039,Old Georgetown Rd & Southwick St,32002,Bethesda Ave & Arlington Rd,W22202,Member
2,715,2016-01-01 00:13:52,2016-01-01 00:25:48,31222,New York Ave & 15th St NW,31214,17th & Corcoran St NW,W21427,Member
3,213,2016-01-01 00:15:29,2016-01-01 00:19:03,31506,1st & Rhode Island Ave NW,31509,New Jersey Ave & R St NW,W01294,Member
4,872,2016-01-01 00:16:16,2016-01-01 00:30:49,31041,Prince St & Union St,31048,King St Metro South,W22058,Member


In [4]:
dataframe_membertype = dataframe_tripdataq1.groupby('Member type')['Duration'].count().reset_index()
dataframe_membertype.rename(columns={'Duration':'Total'}, inplace=True)
print('Dataframe Length: ',len(dataframe_membertype))
dataframe_membertype.head()

Dataframe Length:  2


,Member type,Total
0,Casual,84967
1,Member,467432


In [5]:
dataframe_casualclients = dataframe_tripdataq1[dataframe_tripdataq1['Member type'] == 'Casual']
print('Dataframe Length: ',len(dataframe_casualclients))
dataframe_casualclients.head()

Dataframe Length:  84967


,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
5,859,2016-01-01 00:16:18,2016-01-01 00:30:37,31041,Prince St & Union St,31048,King St Metro South,W01122,Casual
11,278,2016-01-01 00:27:26,2016-01-01 00:32:05,31231,14th & D St NW / Ronald Reagan Building,31262,11th & F St NW,W00922,Casual
23,2018,2016-01-01 00:32:51,2016-01-01 01:06:30,31246,M St & Pennsylvania Ave NW,31513,Rhode Island Ave & V St NE,W22120,Casual
35,1285,2016-01-01 00:39:53,2016-01-01 01:01:18,31632,15th & F St NE,31632,15th & F St NE,W21072,Casual
39,1187,2016-01-01 00:43:49,2016-01-01 01:03:36,31258,Lincoln Memorial,31077,Iwo Jima Memorial/N Meade & 14th St N,W20562,Casual


In [6]:
dataframe_memberclients = dataframe_tripdataq1[dataframe_tripdataq1['Member type'] == 'Member'] 
print('Dataframe Length: ',len(dataframe_memberclients))
dataframe_memberclients.head()

Dataframe Length:  467432


,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
0,166,2016-01-01 00:06:58,2016-01-01 00:09:44,31102,11th & Kenyon St NW,31105,14th & Harvard St NW,W01346,Member
1,448,2016-01-01 00:10:20,2016-01-01 00:17:48,32039,Old Georgetown Rd & Southwick St,32002,Bethesda Ave & Arlington Rd,W22202,Member
2,715,2016-01-01 00:13:52,2016-01-01 00:25:48,31222,New York Ave & 15th St NW,31214,17th & Corcoran St NW,W21427,Member
3,213,2016-01-01 00:15:29,2016-01-01 00:19:03,31506,1st & Rhode Island Ave NW,31509,New Jersey Ave & R St NW,W01294,Member
4,872,2016-01-01 00:16:16,2016-01-01 00:30:49,31041,Prince St & Union St,31048,King St Metro South,W22058,Member


In [15]:
dataframe_casual_by_station = dataframe_casualclients.groupby('Start station number')['Member type'].count().reset_index()
dataframe_casual_by_station.rename(columns={'Start station number':'station_id','Member type':'total'}, inplace=True)
# dataframe_casual_by_station.head()

,station_id,total
0,31000,73
1,31001,75
2,31002,225
3,31003,199
4,31004,16


In [14]:
dataframe_member_by_station = dataframe_memberclients.groupby('Start station number')['Member type'].count().reset_index()
dataframe_member_by_station.rename(columns={'Start station number':'station_id','Member type':'total'}, inplace=True)
# dataframe_member_by_station.head()

,station_id,total
0,31000,226
1,31001,294
2,31002,695
3,31003,713
4,31004,657
